In [ ]:
import pandas as pd
from Models import GNB

train_filepath = 'data/iris_train_set.csv'
test_filepath = 'data/iris_test_set.csv'

train_data = pd.read_csv(train_filepath)
test_data = pd.read_csv(test_filepath)



In [ ]:
ytrain = train_data['iris_type']
Xtrain = train_data.drop(['iris_type'], axis=1)
ytest = test_data['iris_type']
Xtest = test_data.drop(['iris_type'], axis=1)

print(Xtrain.shape)
print(ytrain.shape)
print(Xtest.shape)
print(ytest.shape)

In [ ]:
from preprocessing import Scaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler1_ = MinMaxScaler().fit_transform(Xtrain)
scaler1_ = pd.DataFrame(scaler1_, columns = Xtrain.columns)
scaler1_

In [ ]:
scaler1 = Scaler('minmax').fit_transform(Xtrain)
scaler1

In [ ]:
for i in range(scaler1.shape[0]):
    for j in range(scaler1.shape[1]):

        if round(scaler1.iloc[i,j],10) != round(scaler1_.iloc[i,j],10) :
            print(i,j)

In [ ]:
scaler2 = Scaler('t-distribution').fit_transform(Xtrain)
scaler2

In [ ]:
for col in Xtrain.columns:
    print(scaler1[col].max(), scaler1[col].min())
    print(scaler1_[col].max(), scaler1_[col].min())
# for col in Xtrain.columns:
#     print(scaler2[col].mean(), scaler2[col].std())

In [ ]:
gnb.fit(Xtrain, ytrain)

In [ ]:
ypred = gnb.predict(Xtest)

In [ ]:
gnb.calc_accuracy(ypred, ytest)

In [ ]:
from Models import KNN
knn = KNN(3,'euclidean','t-disribution')

In [ ]:
knn.fit(Xtrain, ytrain)

In [ ]:
pred = knn.predict(Xtest)

In [ ]:
knn.calc_accuracy(ypred, ytest)

In [ ]:
from Models import KMeans
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
centers = 5

X_train, true_labels = make_blobs(n_samples=100, centers=centers, random_state=42)
X_train = StandardScaler().fit_transform(X_train)
sns.scatterplot(x=[X[0] for X in X_train],
                y=[X[1] for X in X_train],
                hue=true_labels,
                palette="deep",
                legend=None
                )
plt.xlabel("x")
plt.ylabel("y")
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=centers, init='kmeans++')
kmeans.fit(X_train)
# View results
class_centers, classification = kmeans.evaluate()
sns.scatterplot(x=[X[0] for X in X_train],
                y=[X[1] for X in X_train],
                hue=true_labels,
                style=classification,
                palette="deep",
                legend=None
                )
plt.plot([x for x, _ in kmeans.centroids],
         [y for _, y in kmeans.centroids],
         '+',
         markersize=10,
         )
plt.show()

In [8]:
import numpy as np
a = [1,2,3,5]
np.not_equal(a,a).any()

False

In [1]:
import numpy as np
from Models import Perceptron
inputs = [1,10,5,46,9]
weights = np.random.rand(5)
bias = np.random.randint(0,10)
print(inputs, weights, bias)

[1, 10, 5, 46, 9] [0.9899584  0.70825256 0.08142009 0.98639958 0.91578403] 1


In [2]:
p = Perceptron(inputs, weights, bias, "sign", learning_const=0.1)
str(p)

'Perceptron: 1'

In [3]:
p.gradient_descent(0)
p.weights

array([ 0.7899584 , -1.29174744, -0.91857991, -8.21360042, -0.88421597])

In [14]:
print(p)

Perceptron: 0


In [2]:
import numpy as np
a = np.random.rand(5)
a

array([0.36651604, 0.59521491, 0.06397428, 0.58324802, 0.78387139])

In [6]:
(b *a).sum()

30

In [7]:
print(a, b)
a = a+ 0*b
print(a, b)


[1 2 3 4] [1 2 3 4]
[1 2 3 4] [1 2 3 4]


In [10]:
random_weights = np.random.rand(5)
random_weights = [x if np.random.randint(4)!=2 else -x for x in random_weights]
random_weights

[0.8255513665302354,
 0.7363270805309007,
 0.07330248955289431,
 0.6344876193372493,
 0.8768269735023101]

In [12]:
a = np.array(None)
a

array(None, dtype=object)

In [1]:
from Models import ANN
a = ANN((2,2), "sigmoid", 5)

In [2]:
str(a)

'Perceptron: na Perceptron: na \nPerceptron: na Perceptron: na \n'